In [1]:
!pip install seaborn
!pip install matplotlib

In [2]:
%load_ext autoreload
%autoreload 2
import sys
sys.path.append('../../')

import sys, importlib
module_name = "Functions"
if module_name in sys.modules:
    del sys.modules[module_name]
Functions = importlib.import_module(module_name)

from Functions_HuggingFace import *

import esm
from Bio import SeqIO
from Bio.Seq import Seq
import pandas as pd 
import numpy as np
import torch

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from Bio import SeqIO

from Bio import Entrez
from Bio import SeqIO

from transformers import EsmForMaskedLM, T5EncoderModel,T5Tokenizer,DataCollatorForLanguageModeling
from transformers.modeling_outputs import MaskedLMOutput


# Run code

In [3]:


model_raw, alphabet = esm.pretrained.load_model_and_alphabet('esm2_t36_3B_UR50D')
model_raw.eval()
batch_converter = alphabet.get_batch_converter()
modnam="/home3/oml4h/hugging_face_downloads/model_weights_topublish/ESM2-H3"
model = EsmForMaskedLM.from_pretrained(modnam)


device = torch.device("cpu")
if torch.cuda.is_available():
    model =  model.to(device)
    print("Transferred model to GPU")

Transferred model to GPU


In [4]:
# Entrez.email = "sample@example.org"

# handle = Entrez.efetch(db="nucleotide",
#                        id="NC_045512.2",
#                        rettype="gb",
#                        retmode="gb")
# whole_sequence = SeqIO.read(handle, "genbank")
model_layers = 36

In [5]:
sequences = read_sequences_to_dict('/home2/oml4h/PLM_SARS-CoV-2/Sequences/huH3N2_HA_CDS.translated.fas')
sequences

{'EPI2178977|HA|A/Thailand/8/2022|EPI_ISL_14991375|J': 'MKAIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQISGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNICIX',
 'EPI2096148|HA|A/Massachusetts/18/2022|EPI_ISL_13897082|J': 'MKAIIALSNILCLVFAQKIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKNESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQN

In [6]:
ids=list(sequences.keys())




In [7]:
J_indexed_muts = [m for m in get_reference_mutations(ref = sequences[ids[0]],mut = sequences[ids[len(ids)-1]]) if "-" not in m  ] 
K_indexed_muts = [m for m in get_mutations(sequences[ids[0]],sequences[ids[len(ids)-1]]) if "del" not in m and '-' not in m  ] 
print("K->J mutations")
print(K_indexed_muts)

K->J mutations
['K18N', 'N138D', 'T151K', 'S160N', 'N174D', 'I176K', 'Q189R', 'K205R', 'K292E', 'T344A', 'S394N']


In [8]:
mutation_dictionary  = {k:J_indexed_muts[i] for i,k in enumerate(K_indexed_muts)}

In [9]:
# get info on model layers:
model_layers_info = model.config.to_dict()
print(model_layers_info)
#print how many layers the model has
print("Number of layers in the model: ", model_layers_info['num_hidden_layers'] + 1)  # +1 for the embedding layer

{'return_dict': True, 'output_hidden_states': False, 'output_attentions': False, 'torchscript': False, 'torch_dtype': 'float32', 'use_bfloat16': False, 'tf_legacy_loss': False, 'pruned_heads': {}, 'tie_word_embeddings': True, 'chunk_size_feed_forward': 0, 'is_encoder_decoder': False, 'is_decoder': False, 'cross_attention_hidden_size': None, 'add_cross_attention': False, 'tie_encoder_decoder': False, 'max_length': 20, 'min_length': 0, 'do_sample': False, 'early_stopping': False, 'num_beams': 1, 'num_beam_groups': 1, 'diversity_penalty': 0.0, 'temperature': 1.0, 'top_k': 50, 'top_p': 1.0, 'typical_p': 1.0, 'repetition_penalty': 1.0, 'length_penalty': 1.0, 'no_repeat_ngram_size': 0, 'encoder_no_repeat_ngram_size': 0, 'bad_words_ids': None, 'num_return_sequences': 1, 'output_scores': False, 'return_dict_in_generate': False, 'forced_bos_token_id': None, 'forced_eos_token_id': None, 'remove_invalid_values': False, 'exponential_decay_length_penalty': None, 'suppress_tokens': None, 'begin_supp

In [10]:

# #pd.DataFrame(columns=["Mutation","rel_grammar","rel_seq_grammar","semanatic_score","probability","Backbone"])

# #loop through each node in the tree except the last one (lineage of interest)
# for backbone_i in range(len(ids)-1):
#     backbone = ids[backbone_i]
#     print("Calculating mutations on backbone:",backbone)
#     reference_spike_sequence = sequences[backbone]
#     #calculate mutation differences between node and K lineage
#     J_indexed_muts = [m for m in get_reference_mutations(ref = sequences[backbone],mut = sequences[ids[len(ids)-1]]) if "-" not in m  ] 
#     K_indexed_muts = [m for m in get_mutations(sequences[backbone],sequences[ids[len(ids)-1]]) if "del" not in m and '-' not in m  ] 
#     mutation_dictionary  = {k:J_indexed_muts[i] for i,k in enumerate(K_indexed_muts)}
#     print("K->J mutations")
#     print(K_indexed_muts)
#     #loop through each mutation and calculate the scores on the focal node
#     for mut in mutation_dictionary.keys():
#         if backbone ==  ids[len(ids)-1]:
#             sequence = revert_sequence(reference_spike_sequence,[mut])
#         else:
#             sequence = mutate_sequence(reference_spike_sequence,[mut])
#         reference = reference_spike_sequence
#         mutations = embed_protein_sequences(
#             [[mut,sequence.replace("-","")]],
#             reference.replace("-",""),
#             'S:0',
#             model,
#             model_layers,
#             device,
#             batch_converter,
#             alphabet,
#             scores=True)
   
        
#         #append mutation info to dataframe
#         new_row={"Mutation":mut,"rel_grammar":mutations[mut]["S:0"]["relative_grammaticality"],
#                                     "rel_seq_grammar":mutations[mut]["S:0"]["relative_sequence_grammaticality"],
#                                     "narrow_seq_grammar":mutations[mut]["S:0"]["narrow_sequence_grammaticality"],
#                                     "relative_narrow_seq_grammar":mutations[mut]["S:0"]["relative_narrow_sequence_grammaticality"],
#                                     "semanatic_score":mutations[mut]["S:0"]["semantic_score"],
#                                     "probability":mutations[mut]["S:0"]["probability"],
#                                     "Backbone":backbone}
#         mut_info_rows.append(new_row)
#         all_mutations[mut] = mutations
    
# mut_info=pd.DataFrame(mut_info_rows)

In [11]:
# #
# mut_info["lineage_backbone"]=mut_info["Backbone"].str.split("|").str[-1]
# mut_info
# mut_info.to_csv("/home2/oml4h/PLM_SARS-CoV-2/Results/H3_epistasis_mutation_info_spyros_model.csv",index=False)
# print(mut_info.loc[mut_info['Mutation']=='I176K'])
# print(mut_info.loc[mut_info['Mutation']=='I151K'])

# do each mutation by itself and then do each pairwise comparison 

In [12]:
J_indexed_muts = [m for m in get_reference_mutations(ref = sequences[ids[0]],mut = sequences[ids[len(ids)-1]]) if "-" not in m  ] 
K_indexed_muts = [m for m in get_mutations(sequences[ids[0]],sequences[ids[len(ids)-1]]) if "del" not in m and '-' not in m  ] 
print("K->J mutations")
print(K_indexed_muts)



K->J mutations
['K18N', 'N138D', 'T151K', 'S160N', 'N174D', 'I176K', 'Q189R', 'K205R', 'K292E', 'T344A', 'S394N']


In [15]:
# backbone = "Wuhan-Hu-1"

backbone=ids[0]
reference_spike_sequence = sequences[backbone]

#create dict of point mutation backbones for all of the mutations in  K_indexed_muts, with names as keys
point_mutations={}
for mut in K_indexed_muts:
    point_mutations[mut]=mutate_sequence(reference_spike_sequence,[mut])

mut_info_rows=[]

#pd.DataFrame(columns=["Mutation","rel_grammar","rel_seq_grammar","semanatic_score","probability","Backbone"])


#loop through each point mutation
for backbone_i in range(len(K_indexed_muts)+1):
    
    if backbone_i == len(K_indexed_muts):
        backbone = reference_spike_sequence
        backbone_name="Reference"
    else:
        backbone = point_mutations[list(point_mutations.keys())[backbone_i]]
        backbone_name=list(point_mutations.keys())[backbone_i]
    
    for mutation_j in range(len(K_indexed_muts)):
        if backbone_i == mutation_j:
            print(f"Skipping mutation {K_indexed_muts[mutation_j]} on its own backbone")
            continue
        mutate_sequence(reference_spike_sequence,[mut])
        
        mut=K_indexed_muts[mutation_j]
     

        sequence = mutate_sequence(backbone,[mut])

        reference = backbone
        mutations = embed_protein_sequences(
            [[mut,sequence.replace("-","")]],
            backbone.replace("-",""),
            'S:0',
            model,
            model_layers,
            device,
            batch_converter,
            alphabet,
            scores=True)

        
        #append mutation info to dataframe
        new_row={"Mutation":mut,"rel_grammar":mutations[mut]["S:0"]["relative_grammaticality"],
                                    "rel_seq_grammar":mutations[mut]["S:0"]["relative_sequence_grammaticality"],
                                    "narrow_seq_grammar":mutations[mut]["S:0"]["narrow_sequence_grammaticality"],
                                    "relative_narrow_seq_grammar":mutations[mut]["S:0"]["relative_narrow_sequence_grammaticality"],
                                    "semanatic_score":mutations[mut]["S:0"]["semantic_score"],
                                    "probability":mutations[mut]["S:0"]["probability"],
                                    "Backbone": backbone_name}
        mut_info_rows.append(new_row)
        
    
mut_info_combos=pd.DataFrame(mut_info_rows)

Skipping mutation K18N on its own backbone
[['N138D', 'MKAIIALSNILCLVFAQNIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKDESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGMVDGWYGFRHQNSEGRGQAADLKSTQAAIDQISGKLNRLIGKTNEKFHQIEKEFSEVEGRVQDLEKYVEDTKIDLWSYNAELLVALENQHTIDLTDSEMNKLFEKTKKQLRENAEDMGNGCFKIYHKCDNACIGSIRNETYDHNVYRDEALNNRFQIKGVELKSGYKDWILWISFAMSCFLLCIALLGFIMWACQKGNIRCNICIX']]
[['N138D', 'MKAIIALSNILCLVFAQNIPGNDNSTATLCLGHHAVPNGTIVKTITNDRIEVTNATELVQNSSIGKICNSPHQILDGGNCTLIDALLGDPQCDGFQNKEWDLFVERSRANSSCYPYDVPDYASLRSLVASSGTLEFKDESFNWTGVKQNGTSSACKRGSSSSFFSRLNWLTSLNNIYPAQNVTMPNKEQFDKLYIWGVHHPDTDKNQFSLFAQSSGRITVSTKRSQQAVIPNIGSRPRVRDIPSRISIYWTIVKPGDILLINSTGNLIAPRGYFKIRSGKSSIMRSDAPIGKCKSECITPNGSIPNDKPFQNVNRITYGACPRYVKQSTLKLATGMRNVPEKQTRGIFGAIAGFIENGWEGM

In [20]:
mut_info_combos.head()

,Mutation,rel_grammar,rel_seq_grammar,narrow_seq_grammar,relative_narrow_seq_grammar,semanatic_score,probability,Backbone,rel_reference_grammar
0,N138D,-5.489066,-1.610174,-22.007191,0.363197,1.954949,0.004104,K18N,0.002149
1,T151K,-6.509132,-0.573269,-22.544548,-0.170448,2.495206,0.001485,K18N,0.018128
2,S160N,-4.929284,-0.250839,-22.027584,0.338257,7.392962,0.007151,K18N,-0.003515
3,N174D,-9.392246,-5.931194,-22.057961,0.318489,3.785834,0.000083,K18N,0.018744
4,I176K,-8.186225,-2.044010,-22.095127,0.275314,7.536953,0.000277,K18N,-0.108524


In [17]:
mut_info_combos["rel_reference_grammar"]=mut_info_combos["rel_grammar"]-mut_info_combos.merge(mut_info_combos[mut_info_combos['Backbone']=="Reference"][['Mutation','rel_grammar']],on='Mutation',how='left')['rel_grammar_y']

In [22]:
#print most shifted mutations compared to reference
mut_info_combos.sort_values(by="rel_reference_grammar",ascending=False).head(10)

,Mutation,rel_grammar,rel_seq_grammar,narrow_seq_grammar,relative_narrow_seq_grammar,semanatic_score,probability,Backbone,rel_reference_grammar
53,S160N,-3.873441,0.158743,-22.694468,0.273783,8.292737,0.020239,I176K,1.052328
74,N174D,-8.820124,-5.093124,-22.273735,0.340631,3.920496,0.000148,K205R,0.590867
76,Q189R,-9.282513,-3.834000,-22.515438,0.098364,6.370613,0.000093,K205R,0.358307
47,K292E,-8.127913,-2.487532,-26.672991,0.465141,3.938951,0.000292,N174D,0.282310
54,N174D,-9.140905,-6.341356,-23.374287,-0.379833,3.800823,0.000107,I176K,0.270085
45,Q189R,-9.380648,-3.466612,-26.246639,0.900951,5.850397,0.000084,N174D,0.260172
22,S160N,-4.759949,0.131973,-20.952396,0.684166,6.774719,0.008458,T151K,0.165820
39,S394N,0.956337,1.246090,-19.905357,-0.038458,4.607357,0.719610,S160N,0.154565
64,N174D,-9.258336,-4.923832,-25.049038,0.642387,4.315244,0.000095,Q189R,0.152655
16,K205R,-7.118632,-1.364687,-22.799536,-0.086584,7.682598,0.000809,N138D,0.127333


In [ ]:
megatable = []

for k in mutation_dictionary.keys():
    if k == 'Reference':
        continue
    relative_grammaticality = all_mutations[k][k]['S:0']["relative_grammaticality"]
    relative_sequence_grammaticality = all_mutations[k][k]['S:0']["relative_sequence_grammaticality"]
    semantic_score = all_mutations[k][k]['S:0']["semantic_score"]

    mutations = pd.DataFrame({'mutation':k,
                            'relative_grammaticality':relative_grammaticality,
                            'relative_sequence_grammaticality':relative_sequence_grammaticality,
                            'semantic_score':semantic_score,},index=[0])
    mutations.mutations = k
    megatable.append(mutations)
megatable_df = pd.concat(megatable)

if backbone == ids[len(ids)-1]:
    for mut in  mutation_dictionary.keys():
        megatable_df.mutation = megatable_df.mutation.str.replace(mut,mutation_dictionary[mut])
megatable_df

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler((0,1))

megatable_df['scaled_relative_grammaticality'] = mms.fit_transform(-megatable_df.relative_grammaticality.values.reshape(-1,1)).ravel()
megatable_df['scaled_relative_sequence_grammaticality'] = mms.fit_transform(-megatable_df.relative_sequence_grammaticality.values.reshape(-1,1)).ravel()

In [ ]:
megatable_df.sort_values('relative_grammaticality',ascending=False)

In [ ]:
import torch
lsoftmax = torch.nn.LogSoftmax(dim=1)

In [ ]:
def outside_std(column):
    filtered_vals = []
    for c in column:
        if c >=(column.mean() + (column.std()*2)) or c <= (column.mean() - (column.std()*2)):
            filtered_vals.append(c)
        else:
            filtered_vals.append(np.nan)
    return filtered_vals


In [ ]:
reference_logits_table = []
for k in all_mutations.keys():
    print(all_mutations[k].keys())
    backbone_logits = all_mutations[k]['Reference']['S:0']['Logits']
    backbone_logits = format_logits(backbone_logits,alphabet)
    backbone_logits = remap_logits(backbone_logits,sequences['Wuhan'], reference_spike_sequence)
    backbone_ref_seq_logits = [row[row.Sequence] if row.Sequence != '-' else np.nan for i,row in backbone_logits.iterrows()]

    for mutation in all_mutations[k].keys():
        if mutation != 'Reference':
            if backbone == 'BA.1':
                mutated_sequence = revert_sequence(reference_spike_sequence,[mutation])
            else:
                mutated_sequence = mutate_sequence(reference_spike_sequence,[mutation])

            mutated_logits = format_logits(all_mutations[k][mutation]['S:0']['Logits'],alphabet)
            mutated_logits = remap_logits(mutated_logits,sequences['Wuhan'], mutated_sequence, )
            mutated_seq_logits = [row[row.Sequence]  if row.Sequence != '-' else np.nan for i,row in mutated_logits.iterrows()]
            
            if backbone == 'BA.1':
                # Difference is backbone - mutation here since the "mutation" is actually a reversion
                difference = np.exp(backbone_ref_seq_logits) -  np.exp(mutated_seq_logits)
                pllr =  np.array(backbone_ref_seq_logits)- np.array(mutated_seq_logits)
            else:
                difference =  np.exp(mutated_seq_logits) - np.exp(backbone_ref_seq_logits)
                pllr =  np.array(mutated_seq_logits) - np.array(backbone_ref_seq_logits)

            df = pd.DataFrame(
                {'source_sequence':list(backbone_logits.Sequence),
                'target_sequence':list(mutated_logits.Sequence),
                'source_logits':backbone_ref_seq_logits,
                'target_logits':mutated_seq_logits,
                'change':difference,
                'ratio':pllr,
                'position':range(1,len(difference)+1)})
                
            df['masked_change'] =[row.change if row.source_sequence == row.target_sequence else np.nan for i, row in df.iterrows()]
            df['masked_ratio'] =[row.ratio if row.source_sequence == row.target_sequence else np.nan for i, row in df.iterrows()]
            
            local_max_change = df.masked_change.mean()+(df.masked_change.std()*2)
            local_min_change = df.masked_change.mean()-(df.masked_change.std()*2)

            local_max_ratio = df.masked_ratio.mean()+(df.masked_ratio.std()*2)
            local_min_ratio = df.masked_ratio.mean()-(df.masked_ratio.std()*2)

            df['significant_masked_change_local'] = [ check_valid(m,local_min_change,local_max_change) for m in df. masked_change]
            df['significant_masked_ratio_local'] = [ check_valid(m,local_min_ratio,local_max_ratio) for m in df.masked_ratio]

            df['mutation'] = mutation_dictionary[mutation]
            reference_logits_table.append(df)
reference_logits_table = pd.concat(reference_logits_table,axis=0)

max_change = reference_logits_table.masked_change.mean()+(reference_logits_table.masked_change.std()*2)
min_change = reference_logits_table.masked_change.mean()-(reference_logits_table.masked_change.std()*2)

max_ratio = reference_logits_table.masked_ratio.mean()+(reference_logits_table.masked_ratio.std()*2)
min_ratio = reference_logits_table.masked_ratio.mean()-(reference_logits_table.masked_ratio.std()*2)

reference_logits_table['significant_masked_change'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change]
reference_logits_table['significant_masked_ratio'] = [ check_valid(m,min_ratio,max_ratio) for m in reference_logits_table.masked_ratio]

max_change = reference_logits_table.masked_change.round(4).mean()+(reference_logits_table.masked_change.round(4).std()*2)
min_change = reference_logits_table.masked_change.round(4).mean()-(reference_logits_table.masked_change.round(4).std()*2)

reference_logits_table['significant_masked_change_rounded'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change.round(4)]


In [ ]:
import plotly.express as px
fig = px.line(reference_logits_table, x="position", y="masked_change",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
import plotly.express as px
fig = px.scatter(reference_logits_table, x="position", y="significant_masked_change_rounded",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
reference_logits_table.to_csv('./Epistasis/Revisions_Omicron_Epistasis.csv',index=False)

In [ ]:
reference_logits_table = pd.read_csv('./Epistasis/Revisions_Omicron_Epistasis.csv')

In [ ]:
max_change = reference_logits_table.masked_change.mean()+(reference_logits_table.masked_change.std())
min_change = reference_logits_table.masked_change.mean()-(reference_logits_table.masked_change.std())
reference_logits_table['single_significant_masked_change'] = [ check_valid(m,min_change,max_change) for m in reference_logits_table.masked_change]

In [ ]:
reference_logits_table.to_csv('Revisions_Omicron_Epistasis_with_single_std.csv')

In [ ]:
import plotly.express as px
fig = px.scatter(reference_logits_table, x="position", y="single_significant_masked_change",color="mutation", facet_col='mutation',facet_col_wrap=6,height=1000, width=1500, hover_data=['position','change','mutation'])
fig.update_traces(marker={'size': 3})
fig.show()

In [ ]:
ba1_mutated_positions = reference_logits_table.mutation.str[1:-1].astype(int).unique()
ba1_mutated_positions

In [ ]:
one_std_identified_interactions = reference_logits_table[(reference_logits_table.single_significant_masked_change.isna() == False) & (reference_logits_table.position.isin(ba1_mutated_positions)) ]

In [ ]:
two_std_identified_interactions = reference_logits_table[(reference_logits_table.significant_masked_change.isna() == False) & (reference_logits_table.position.isin(ba1_mutated_positions)) ]

In [ ]:
px.scatter(two_std_identified_interactions,x=two_std_identified_interactions.position.astype(str),y=two_std_identified_interactions.mutation.astype(str))

In [ ]:
px.scatter(one_std_identified_interactions,x=one_std_identified_interactions.position.astype(str),y=one_std_identified_interactions.mutation.astype(str))

In [ ]:
identified_interactions = one_std_identified_interactions[['position','mutation']]
identified_interactions['mutated_position'] = identified_interactions.mutation.str[1:-1].astype(int)
swap_interactions = identified_interactions[identified_interactions.position < identified_interactions.mutated_position]
swap_interactions.rename(columns={'position':'position_1','mutated_position':'position_2'},inplace=True)
swap_interactions_2 = identified_interactions[identified_interactions.position >= identified_interactions.mutated_position]
swap_interactions_2.rename(columns={'mutated_position':'position_1','position':'position_2'},inplace=True)

swap_interactions = pd.concat([swap_interactions,swap_interactions_2])
swap_interactions.to_csv('Omicron_One_Std_Interactions.csv',index=False)

In [ ]:
identified_interactions = two_std_identified_interactions[['position','mutation']]
identified_interactions['mutated_position'] = identified_interactions.mutation.str[1:-1].astype(int)
swap_interactions = identified_interactions[identified_interactions.position < identified_interactions.mutated_position]
swap_interactions.rename(columns={'position':'position_1','mutated_position':'position_2'},inplace=True)
swap_interactions_2 = identified_interactions[identified_interactions.position >= identified_interactions.mutated_position]
swap_interactions_2.rename(columns={'mutated_position':'position_1','position':'position_2'},inplace=True)

swap_interactions = pd.concat([swap_interactions,swap_interactions_2])
swap_interactions.to_csv('Omicron_Two_Std_Interactions.csv',index=False)